# Basic face recognizer using pre trained model

Below the is the implementation of a very basic face recognizer which can identify the face of the person showing on a web cam.
The implementation is inspired by two path breaking papers on facial recognition using deep convoluted neural network, namely FaceNet and DeepFace.

I have used pre trained model Keras-OpenFace which is an open source Keras implementation of the OpenFace (Originally Torch implemented) 

The pretrained model that I have used is by Victor Sy Wang's implementation and was loaded using his code: https://github.com/iwantooxxoox/Keras-OpenFace.

# Import the prerequisite libraries

We will be importing utils.py from https://github.com/iwantooxxoox/Keras-OpenFace/blob/master/utils.py (available with code) which contains utility functions to create the neural network and load the weights assoiated with it.

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.engine.topology import Layer
from keras import backend as K
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from utils import LRN2D
import utils

%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

/home/rana/anaconda3/envs/conaenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Contructing the neural network model
The model here constructed is based on FaceNet's Inception model.

The implementation of model is available at: https://github.com/iwantooxxoox/Keras-OpenFace

In [2]:
myInput = Input(shape=(96, 96, 3))

x = ZeroPadding2D(padding=(3, 3), input_shape=(96, 96, 3))(myInput)
x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn1')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)
x = Lambda(LRN2D, name='lrn_1')(x)
x = Conv2D(64, (1, 1), name='conv2')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn2')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = Conv2D(192, (3, 3), name='conv3')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn3')(x)
x = Activation('relu')(x)
x = Lambda(LRN2D, name='lrn_2')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)

# Inception3a
inception_3a_3x3 = Conv2D(96, (1, 1), name='inception_3a_3x3_conv1')(x)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn1')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)
inception_3a_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3a_3x3)
inception_3a_3x3 = Conv2D(128, (3, 3), name='inception_3a_3x3_conv2')(inception_3a_3x3)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn2')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)

inception_3a_5x5 = Conv2D(16, (1, 1), name='inception_3a_5x5_conv1')(x)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn1')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)
inception_3a_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3a_5x5)
inception_3a_5x5 = Conv2D(32, (5, 5), name='inception_3a_5x5_conv2')(inception_3a_5x5)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn2')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)

inception_3a_pool = MaxPooling2D(pool_size=3, strides=2)(x)
inception_3a_pool = Conv2D(32, (1, 1), name='inception_3a_pool_conv')(inception_3a_pool)
inception_3a_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_pool_bn')(inception_3a_pool)
inception_3a_pool = Activation('relu')(inception_3a_pool)
inception_3a_pool = ZeroPadding2D(padding=((3, 4), (3, 4)))(inception_3a_pool)

inception_3a_1x1 = Conv2D(64, (1, 1), name='inception_3a_1x1_conv')(x)
inception_3a_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_1x1_bn')(inception_3a_1x1)
inception_3a_1x1 = Activation('relu')(inception_3a_1x1)

inception_3a = concatenate([inception_3a_3x3, inception_3a_5x5, inception_3a_pool, inception_3a_1x1], axis=3)

# Inception3b
inception_3b_3x3 = Conv2D(96, (1, 1), name='inception_3b_3x3_conv1')(inception_3a)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn1')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)
inception_3b_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3b_3x3)
inception_3b_3x3 = Conv2D(128, (3, 3), name='inception_3b_3x3_conv2')(inception_3b_3x3)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn2')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)

inception_3b_5x5 = Conv2D(32, (1, 1), name='inception_3b_5x5_conv1')(inception_3a)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn1')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)
inception_3b_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3b_5x5)
inception_3b_5x5 = Conv2D(64, (5, 5), name='inception_3b_5x5_conv2')(inception_3b_5x5)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn2')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)

inception_3b_pool = Lambda(lambda x: x**2, name='power2_3b')(inception_3a)
inception_3b_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: x*9, name='mult9_3b')(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_3b')(inception_3b_pool)
inception_3b_pool = Conv2D(64, (1, 1), name='inception_3b_pool_conv')(inception_3b_pool)
inception_3b_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_pool_bn')(inception_3b_pool)
inception_3b_pool = Activation('relu')(inception_3b_pool)
inception_3b_pool = ZeroPadding2D(padding=(4, 4))(inception_3b_pool)

inception_3b_1x1 = Conv2D(64, (1, 1), name='inception_3b_1x1_conv')(inception_3a)
inception_3b_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_1x1_bn')(inception_3b_1x1)
inception_3b_1x1 = Activation('relu')(inception_3b_1x1)

inception_3b = concatenate([inception_3b_3x3, inception_3b_5x5, inception_3b_pool, inception_3b_1x1], axis=3)

# Inception3c
inception_3c_3x3 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_3x3',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))

inception_3c_5x5 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))

inception_3c_pool = MaxPooling2D(pool_size=3, strides=2)(inception_3b)
inception_3c_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_3c_pool)

inception_3c = concatenate([inception_3c_3x3, inception_3c_5x5, inception_3c_pool], axis=3)

#inception 4a
inception_4a_3x3 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=192,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_4a_5x5 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(1, 1),
                                   padding=(2, 2))

inception_4a_pool = Lambda(lambda x: x**2, name='power2_4a')(inception_3c)
inception_4a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: x*9, name='mult9_4a')(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_4a')(inception_4a_pool)
inception_4a_pool = utils.conv2d_bn(inception_4a_pool,
                                   layer='inception_4a_pool',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   padding=(2, 2))
inception_4a_1x1 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_4a = concatenate([inception_4a_3x3, inception_4a_5x5, inception_4a_pool, inception_4a_1x1], axis=3)

#inception4e
inception_4e_3x3 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_3x3',
                                   cv1_out=160,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))
inception_4e_5x5 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_5x5',
                                   cv1_out=64,
                                   cv1_filter=(1, 1),
                                   cv2_out=128,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))
inception_4e_pool = MaxPooling2D(pool_size=3, strides=2)(inception_4a)
inception_4e_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_4e_pool)

inception_4e = concatenate([inception_4e_3x3, inception_4e_5x5, inception_4e_pool], axis=3)

#inception5a
inception_5a_3x3 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))

inception_5a_pool = Lambda(lambda x: x**2, name='power2_5a')(inception_4e)
inception_5a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: x*9, name='mult9_5a')(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_5a')(inception_5a_pool)
inception_5a_pool = utils.conv2d_bn(inception_5a_pool,
                                   layer='inception_5a_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   padding=(1, 1))
inception_5a_1x1 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))

inception_5a = concatenate([inception_5a_3x3, inception_5a_pool, inception_5a_1x1], axis=3)

#inception_5b
inception_5b_3x3 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_5b_pool = MaxPooling2D(pool_size=3, strides=2)(inception_5a)
inception_5b_pool = utils.conv2d_bn(inception_5b_pool,
                                   layer='inception_5b_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1))
inception_5b_pool = ZeroPadding2D(padding=(1, 1))(inception_5b_pool)

inception_5b_1x1 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_5b = concatenate([inception_5b_3x3, inception_5b_pool, inception_5b_1x1], axis=3)

av_pool = AveragePooling2D(pool_size=(3, 3), strides=(1, 1))(inception_5b)
reshape_layer = Flatten()(av_pool)
dense_layer = Dense(128, name='dense_layer')(reshape_layer)
norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(dense_layer)


# Final Model
model = Model(inputs=[myInput], outputs=norm_layer)

# Loading the model with pretrained weights

FaceNet is trained by minimizing the triplet loss. I will be  loading a previously trained model. weights are avaiable at https://github.com/iwantooxxoox/Keras-OpenFace in the "weights" folder which is also provided in this source.

This can take a couple of minutes to execute and depends on the speed of your machine.


In [3]:
'''
# Load weights from csv files (which was exported from Openface torch model)
weights = utils.weights
weights_dict = utils.load_weights()

# Set layer weights of the model
for name in weights:
  if model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])
  elif model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])
    '''

In [3]:
#model.save_weights("face_model.h5")
model.load_weights("face_model.h5")
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 102, 102, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 48, 48, 64)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 48, 48, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

## About <font color=blue>image_to_embedding</font> function        
When the model is loaded with pre trained weights, then we can create the **128 dimensional embedding vectors** for all the face images stored in the "images" folder. **"image_to_embedding"** function pass an image to the Inception network to generate the embedding vector.

In [4]:
def image_to_embedding(image, model):
    #image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_AREA) 
    image = cv2.resize(image, (96, 96)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding


## About <font color=blue>recognize_face</font> function
This function calculate similarity between the captured image and the images that are already been stored. It passes the image to the trained neural network to generate its embedding vector. Which is then compared with all the embedding vectors of the images stored by calculating L2 Euclidean distance. 

If the minimum L2 distance between two embeddings is less than a threshpld (here I have taken the threashhold as .68 (which can be adjusted) then we have a match.

In [6]:
def recognize_face(face_image, input_embeddings, model):

    embedding = image_to_embedding(face_image, model)
    
    minimum_distance = 200
    name = None
    
    # Loop over  names and encodings.
    for (input_name, input_embedding) in input_embeddings.items():
        
       
        euclidean_distance = np.linalg.norm(embedding-input_embedding)
        

        print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))

        if input_name.find("talha") !=-1:
            input_name="talha"
        if input_name.find("zeeshan") !=-1:
            input_name="zeeshan"
        if input_name.find("hadi") !=-1:
            input_name="hadi"
        if euclidean_distance < minimum_distance:
            minimum_distance = euclidean_distance
            name = input_name
    
    if minimum_distance < 0.68:
        return str(name)
    else:
        return None

## About <font color=blue>create_input_image_embeddings</font> function
This function generates 128 dimensional image ebeddings of all the images stored in the "images" directory by feed forwarding the images to a trained neural network. It creates a dictionary with key as the name of the face and value as embedding


## About <font color=blue>recognize_faces_in_cam</font> function
This function capture image from the webcam, detect a face in it and crop the image to have a face only, which is then passed to recognize_face function. 

In [9]:
import glob

def create_input_image_embeddings():
    input_embeddings = {}

    for file in glob.glob("images/*"):
        person_name = os.path.splitext(os.path.basename(file))[0]
        #if person_name.find("talha") !=-1:
         #   person_name="talha"
        image_file = cv2.imread(file, 1)
        input_embeddings[person_name] = image_to_embedding(image_file, model)
    return input_embeddings

def recognize_faces_in_cam(input_embeddings):
    

    
    vc = cv2.VideoCapture("final.mp4")
    
    cv2.namedWindow("Face Recognizer")
   

    font = cv2.FONT_HERSHEY_SIMPLEX
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    
    while True:
        _, frame = vc.read()
        img = frame
        print(frame)
        height, width, channels = frame.shape

        
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Loop through all the faces detected 
        identities = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h

           
            
            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            identity = recognize_face(face_image, input_embeddings, model)
            
            

            if identity is not None:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)
                cv2.putText(img, str(identity), (x1+5,y1-5), font, 1, (255,255,255), 2)
        
        key = cv2.waitKey(100)
        cv2.imshow("Face Recognizer", img)

        if key == 27: # exit on ESC
            break
    vc.release()
    cv2.destroyAllWindows()
    

In [8]:
embadings=create_input_image_embeddings()

In [24]:
print(embadings)

{'talha83': array([[-2.23117471e-02,  1.21554919e-02,  4.48547862e-02,
         7.46603981e-02,  4.68490981e-02,  2.12877646e-01,
         5.08776587e-03, -8.29283372e-02,  7.27197826e-02,
        -1.44435436e-01, -6.50395220e-03,  2.00698301e-02,
         2.38236487e-02, -7.14273602e-02,  3.24793812e-03,
         1.86755341e-02, -1.19110063e-01,  1.15044508e-02,
        -8.51297677e-02,  4.65318635e-02,  1.93285927e-01,
        -1.15140751e-01,  6.13812124e-03,  1.22866735e-01,
         7.38771483e-02, -1.01857208e-01, -2.05356732e-01,
        -1.98346809e-01, -6.21831417e-02,  5.70132881e-02,
         2.07435071e-01,  4.13794741e-02, -4.91285212e-02,
         7.36091509e-02,  1.10084414e-01,  1.52638525e-01,
        -2.06252392e-02, -1.63433496e-02, -6.90015266e-03,
        -1.11155383e-01,  7.97451884e-02, -5.55799566e-02,
         5.59204929e-02, -2.04557329e-02,  2.69802529e-02,
        -1.09601222e-01,  1.18156530e-01,  6.26379997e-03,
        -7.69241154e-02,  2.24857479e-02, -1

In [ ]:
recognize_faces_in_cam(embadings)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Euclidean distance from hadi52 is 0.6610812
Euclidean distance from hadi61 is 0.48250827
Euclidean distance from talha77 is 0.7032755
Euclidean distance from hadi43 is 0.763266
Euclidean distance from hadi97 is 0.68943447
Euclidean distance from hadi70 is 0.939419
Euclidean distance from hadi49 is 0.7014884
Euclidean distance from talha91 is 0.7456948
Euclidean distance from talha64 is 0.728609
Euclidean distance from hadi57 is 0.5301285
Euclidean distance from talha93 is 0.74540925
Euclidean distance from talha41 is 0.74044603
Euclidean distance from hadi15 is 0.72487485
Euclidean distance from zeeshan6 is 0.85905033
Euclidean distance from talha47 is 0.7470248
Euclidean distance from talha65 is 0.7579875
Euclidean distance from hadi55 is 0.6598299
Euclidean distance from talha100 is 0.7803659
Euclidean distance from zeeshan5 is 0.8051009
Euclidean distance from zeeshan24 is 0.80536574
Euclidean distance from zeeshan12 is 0.8707753
Euclidean distance from hadi66 is 0.72459376
Euclidea

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Euclidean distance from hadi52 is 0.6124955
Euclidean distance from hadi61 is 0.4669388
Euclidean distance from talha77 is 0.78809243
Euclidean distance from hadi43 is 0.7034642
Euclidean distance from hadi97 is 0.6765336
Euclidean distance from hadi70 is 0.88156784
Euclidean distance from hadi49 is 0.64703643
Euclidean distance from talha91 is 0.7615394
Euclidean distance from talha64 is 0.79468906
Euclidean distance from hadi57 is 0.48049748
Euclidean distance from talha93 is 0.7791551
Euclidean distance from talha41 is 0.7132823
Euclidean distance from hadi15 is 0.65357
Euclidean distance from zeeshan6 is 0.88574827
Euclidean distance from talha47 is 0.7150734
Euclidean distance from talha65 is 0.8252036
Euclidean distance from hadi55 is 0.61278194
Euclidean distance from talha100 is 0.8038447
Euclidean distance from zeeshan5 is 0.81308293
Euclidean distance from zeeshan24 is 0.8078576
Euclidean distance from zeeshan12 is 0.88084173
Euclidean distance from hadi66 is 0.66236174
Eucli

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Euclidean distance from hadi52 is 0.66665196
Euclidean distance from hadi61 is 0.56015337
Euclidean distance from talha77 is 0.8239063
Euclidean distance from hadi43 is 0.7375475
Euclidean distance from hadi97 is 0.67004853
Euclidean distance from hadi70 is 0.812532
Euclidean distance from hadi49 is 0.71480995
Euclidean distance from talha91 is 0.7144726
Euclidean distance from talha64 is 0.78747165
Euclidean distance from hadi57 is 0.5067012
Euclidean distance from talha93 is 0.76841897
Euclidean distance from talha41 is 0.7015612
Euclidean distance from hadi15 is 0.6835108
Euclidean distance from zeeshan6 is 0.8590508
Euclidean distance from talha47 is 0.70883393
Euclidean distance from talha65 is 0.81424594
Euclidean distance from hadi55 is 0.6118304
Euclidean distance from talha100 is 0.77998334
Euclidean distance from zeeshan5 is 0.7745734
Euclidean distance from zeeshan24 is 0.72277445
Euclidean distance from zeeshan12 is 0.81170714
Euclidean distance from hadi66 is 0.63946617
Eu

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Euclidean distance from hadi52 is 0.72381884
Euclidean distance from hadi61 is 0.6641944
Euclidean distance from talha77 is 0.9099317
Euclidean distance from hadi43 is 0.7680374
Euclidean distance from hadi97 is 0.72959304
Euclidean distance from hadi70 is 1.0241019
Euclidean distance from hadi49 is 0.67703277
Euclidean distance from talha91 is 0.97969466
Euclidean distance from talha64 is 0.9171667
Euclidean distance from hadi57 is 0.79311496
Euclidean distance from talha93 is 0.9225555
Euclidean distance from talha41 is 0.9457775
Euclidean distance from hadi15 is 0.81592387
Euclidean distance from zeeshan6 is 1.1466049
Euclidean distance from talha47 is 0.9199518
Euclidean distance from talha65 is 0.9259822
Euclidean distance from hadi55 is 0.871278
Euclidean distance from talha100 is 1.0038527
Euclidean distance from zeeshan5 is 1.1057513
Euclidean distance from zeeshan24 is 1.0607026
Euclidean distance from zeeshan12 is 1.1347929
Euclidean distance from hadi66 is 0.89131343
Euclide

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Capturing the face image
Following code captures 10 face images of the person. They all are stored in **"images"** folder with the name User_1 to User_10. Select a good captured image from the set of 10 images. Rename it with the name of person and delete rest of them. This image will be used for recognizing the the identity of the person using one shot learning.

In [17]:
print(embadings)

{'talha': array([[-0.0073184 ,  0.09710307,  0.0739255 ,  0.11024836, -0.0082513 ,
         0.22730763, -0.03609352, -0.1092994 , -0.00503406, -0.12323545,
         0.03472897,  0.04012552, -0.05259926, -0.12375665,  0.00100145,
        -0.07222142, -0.09855896,  0.01389686, -0.01694784,  0.09409071,
         0.02618501, -0.05971131,  0.05271079,  0.09491225,  0.03111317,
        -0.05501433, -0.09313961, -0.08532983, -0.03517501, -0.04391602,
         0.15657681, -0.01020222, -0.02827452,  0.09286058,  0.08549304,
         0.13100778,  0.03313581,  0.02452001, -0.01868108, -0.07393228,
         0.09790815, -0.1055328 ,  0.01377283, -0.04376515, -0.10019749,
        -0.10772042,  0.1249359 ,  0.00569132, -0.09719499,  0.14246824,
        -0.04673095, -0.01141298,  0.02895137,  0.03836491,  0.11139525,
         0.04345188, -0.05985874,  0.05647569, -0.07785788, -0.08414223,
        -0.15585114,  0.08655155,  0.17352681, -0.26946387,  0.15902837,
         0.06028251,  0.06410053, -0.1514

In [1]:
import cv2
cam = cv2.VideoCapture("/home/rana/mypython/FaceRecognition/Face-recognition-using-deep-learning-master/hadi.mp4")

face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

count = 0
count +=1
print(count)
while(True):
    ret, img = cam.read()
    #print(ret)
    #print(img)
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(img, 1.3, 5)
    print(faces)
    for (x,y,w,h) in faces:
        x1 = x
        y1 = y
        x2 = x+w
        y2 = y+h
        cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)     
        count += 1
        print(count)
        # Save the captured image into the datasets folder
        cv2.imwrite("images/hadi" + str(count) + ".jpg", img[y1:y2,x1:x2])
        cv2.imshow('image', img)
    k = cv2.waitKey(200) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
    elif count >= 100: # Take 30 face sample and stop video
         break
cam.release()
cv2.destroyAllWindows()

1
[[148  93 231 231]]
2
[[148  91 234 234]]
3
[[149  89 234 234]]
4
[[150  88 235 235]]
5
[[152  88 234 234]]
6
[[152  85 238 238]]
7
[[154  83 240 240]]
8
[[155  83 242 242]]
9
[[154  79 247 247]]
10
[[154  83 239 239]]
11
[[157  85 236 236]]
12
[[158  84 241 241]]
13
[[158  83 243 243]]
14
[[159  79 248 248]]
15
[[162  79 244 244]]
16
[[167  81 241 241]]
17
[[169  84 241 241]]
18
[[173  81 243 243]]
19
[[174  79 248 248]]
20
[[177  79 248 248]]
21
[[178  76 255 255]]
22
[[184  82 245 245]]
23
[[185  78 250 250]]
24
[[187  78 252 252]]
25
[[195  80 247 247]]
26
[[193  77 252 252]]
27
[[195  77 255 255]]
28
[[204  81 249 249]]
29
[[207  82 250 250]]
30
[[214  85 245 245]]
31
[[212  78 257 257]]
32
[[213  78 257 257]]
33
[[214  74 264 264]]
34
[[220  80 256 256]]
35
[[224  83 251 251]]
36
[[226  82 252 252]]
37
[[222  78 261 261]]
38
[[226  82 255 255]]
39
[[223  83 255 255]]
40
[[233  82 255 255]]
41
[[239  81 255 255]]
42
[[239  80 251 251]]
43
[[238  81 255 255]]
44
[[238  76 260 260

# Lets run the face recognizer program :-)

In [10]:
input_embeddings = create_input_image_embeddings()


In [24]:
def recognize_faces_in_cam(input_embeddings):
    
    
   

    font = cv2.FONT_HERSHEY_SIMPLEX
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    
    frame =cv2.imread("l2.jpg")
    img = frame
    print(frame)
    height, width, channels = frame.shape

        
        
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Loop through all the faces detected 
    identities = []
    for (x, y, w, h) in faces:
        x1 = x
        y1 = y
        x2 = x+w
        y2 = y+h

           
            
    face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
    identity = recognize_face(face_image, input_embeddings, model)
    #print(identify)
            
            

    if identity is not None:
        img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)
        cv2.putText(img, str(identity), (x1+5,y1-5), font, 1, (255,255,255), 2)
        
    cv2.imwrite("rana.jpg",img)
    cv2.destroyAllWindows()

In [11]:
recognize_faces_in_cam(input_embeddings)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
!conda list

# packages in environment at /home/rana/anaconda3/envs/conaenv:
#
# Name                    Version                   Build  Channel
absl-py                   0.4.1                    py35_0  
alabaster                 0.7.10           py35h6fb19ab_0  
anaconda                  custom           py35h104c396_0  
anaconda-client           1.6.14                   py35_0  
anaconda-project          0.8.2            py35h7d62c80_0  
asn1crypto                0.24.0                   py35_0  
astor                     0.7.1                    py35_0  
astroid                   1.6.3                    py35_0  
astropy                   3.0.2            py35h3010b51_1  
attrs                     18.1.0                   py35_0  
babel                     2.5.3                    py35_0  
backcall                  0.1.0                    py35_0  
backports                 1.0              py35hd471ac7_1  
backports.shutil_get_terminal_size 1.0.0            py35h40844db_2  
backports.weakref 